In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf



In [6]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.6 MB/s eta 0:00:00


In [4]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [5]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [7]:
#Import IMDB Training set from Stanford.
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')


84125825/84125825 [==============================] - 3s 0us/step


In [12]:
# The shutil module offers a number of high-level 
# operations on files and collections of files.
import os
import shutil
# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
# View the final train folder
print(os.listdir(train_dir))

['urls_pos.txt', 'neg', 'urls_neg.txt', 'urls_unsup.txt', 'unsupBow.feat', 'pos', 'labeledBow.feat']


In [13]:
# We create a training dataset and a validation 
# dataset from our "aclImdb/train" directory with a 80/20 split.
train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='training', seed=123)
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='validation', seed=123)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [15]:
#Convert data to Pandas to view and process for Training Data
for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
train.head()

,DATA_COLUMN,LABEL_COLUMN
0,Canadian director Vincenzo Natali took the art...,1
1,I gave this film 10 not because it is a superb...,1
2,I admit to being somewhat jaded about the movi...,1
3,"For a long time, 'The Menagerie' was my favori...",1
4,A truly frightening film. Feels as if it were ...,0


In [16]:
#Convert Test data also to Data Frame for processing and vieweing
for i in test.take(1):
    test_feat = i[0].numpy()
    test_lab = i[1].numpy()
    
test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test.head()
    



,DATA_COLUMN,LABEL_COLUMN
0,I can't believe that so much talent can be was...,0
1,This movie blows - let's get that straight rig...,0
2,"The saddest thing about this ""tribute"" is that...",0
3,I'm only rating this film as a 3 out of pity b...,0
4,Something surprised me about this movie - it w...,1


In [18]:
#Method to convert Train and Test Data sets to format that BERT needs

def convert_data(train, test, DATA_COLUMN, LABEL_COLUMN): 
    train_Input = train.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
    validation_Input = test.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
    return train_Input, validation_Input

train_Input, validation_Input = convert_data(train, test, 'DATA_COLUMN', 'LABEL_COLUMN')

    

In [19]:
#Tokenizing Data and creating Features

def convert_examples_to_tf(examples, tokenizer, max_length=256):
    features = []

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'


In [20]:
#Calling above functions to get the data for BERT Model
train_Input, validation_Input = convert_data(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf(list(train_Input), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf(list(validation_Input), tokenizer)
validation_data = validation_data.batch(32)

In [21]:
validation_data

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [22]:
#compile and train the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
1250/1250 [==============================] - 310s 227ms/step - loss: 0.2161 - accuracy: 0.9120 - val_loss: 0.2466 - val_accuracy: 0.9116
Epoch 2/2
1250/1250 [==============================] - 281s 225ms/step - loss: 0.0589 - accuracy: 0.9815 - val_loss: 0.3500 - val_accuracy: 0.9110


In [23]:
model.save("/content/aclimdb256/Model", save_format='tf')

In [25]:
#Making Predictions
pred_sentences = ['Movie was great. It had a gripping plot and a nice ending!!',
                  'Very boring Movie. Not much to write about :(']

#Tokenizing the test sentences
tf_batch = tokenizer(pred_sentences, max_length=256, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

Movie was great. It had a gripping plot and a nice ending!! : 
 Positive
Very boring Movie. Not much to write about :( : 
 Negative


In [27]:
#Making Predictions
pred_sentences = test.DATA_COLUMN.values[1:1000].tolist()

#Tokenizing the test sentences
tf_batch = tokenizer(pred_sentences, max_length=256, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
#tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
tf_predictions = tf.nn.softmax(tf_outputs['logits'], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(10):
  print(pred_sentences[i], ": \n", labels[label[i]])


This movie blows - let's get that straight right now. There are a few scene gems nestled inside this pile of crap but none can redeem the limp plot. Colin Farrel looks like Brad Pitt in "12 Monkeys" and acts in a similar manner. I normally hate Colin because he is a fairy in general but he's OK in this movie. There were two plot lines in this movie-= one about a kid who throws rocks through windshields of moving vehicles and the other about a woman with a moustache. Let's face it- this movie has no freaking idea of what it wanted to say or where it wanted to go. THe characters story lines intertwine on some levels but are in no means worthy of being included in a script. The whole thing is weak and pointless and then there is an occasional OK scene. But overall- Don't bother unless you love irish accents so much that you can watch mediocrity and it is rescued by everyone sounding like the Lucky Charms elf -an American fetish that has catapulted some truly crappy movies to success. : 
 

In [29]:
#Getting Precision
actual_labels = test.LABEL_COLUMN.values[1:1000].tolist()
predicted_labels = label
m = tf.keras.metrics.Precision()
m.update_state(actual_labels, predicted_labels)
m.result().numpy()

0.91793895

In [30]:
#Getting Recall
m = tf.keras.metrics.Recall()
m.update_state(actual_labels, predicted_labels)
m.result().numpy()

0.9303675